In [1]:
def obtenerDia(time):     # Funcion para corregir fechas de softw. OPC/Alphas.
    fecha_reference = datetime(1900 , 1 , 1)
    fecha_final = fecha_reference + timedelta(days = int(time)-2)

    return str(fecha_final)[0:10]

def obtenerHora(sec):
    horas = sec//3600
    res = sec%3600
    minutos =  res//60
    segundos = res%60
    total_h = []

    if horas < 10:
        extra_h = '0'
    else:
        extra_h = ''

    if minutos < 10:
        extra_m = '0'
    else:
        extra_m = ''

    if segundos < 10:
        extra_s = '0'
    else:
        extra_s = ''
        
    total_h.append(extra_h + str(horas) + ':' + extra_m + str(minutos) + ':' + extra_s + str(segundos))
    return total_h[0]  

def obtenerFecha(time,sec):
    return obtenerDia(time) + ' ' + obtenerHora(sec)

def corregirFecha(df,column):
    time = df.OADateTime
    time_d = time.astype(int)
    time_h = time - time_d
    segundos = 24*3600*time_h
    sec = segundos.astype(int)
    fechas = []
    
    for i in range(0,len(time_d)):
        fechas.append(obtenerFecha(time_d[i],sec[i]))
    fechas = pd.to_datetime(pd.Series(fechas))

    df[column] = fechas
    return df
# -----------------------------------------------------------------------------------------------

def PL(x,sfr):                 # Funcion que pasa de p/s a p/L en funcion del sampleflowrate
    return x*((10**3)/sfr) 
# -----------------------------------------------------------------------------------------------

def ps_to_pl(bins, S):  # Funcion para convertir una columna del bin de p/s a p/l en funcion de sfr(S)
    b= []        
    for i in range(len(bins)):
        a = PL(bins[i] , S[i] )
        b.append(a)
    return b
# -------------------------------------------------------------------------------------------

def new_opc_pl(df):    # Funcion que convierte el df del opc en part/L y devuelve otro df
    S= df['SFR(ml/s)']
    df_opc = pd.DataFrame()
    
    df_opc['date'] = df.index
    df_opc['Bin00'] = ps_to_pl(df['Bin00'] , S)
    df_opc['Bin01'] = ps_to_pl(df['Bin01'] , S)
    df_opc['Bin02'] = ps_to_pl(df['Bin02'] , S)
    df_opc['Bin03'] = ps_to_pl(df['Bin03'] , S)
    df_opc['Bin04'] = ps_to_pl(df['Bin04'] , S)
    df_opc['Bin05'] = ps_to_pl(df['Bin05'] , S)
    df_opc['Bin06'] = ps_to_pl(df['Bin06'] , S)
    df_opc['Bin07'] = ps_to_pl(df['Bin07'] , S)
    df_opc['Bin08'] = ps_to_pl(df['Bin08'] , S)
    df_opc['Bin09'] = ps_to_pl(df['Bin09'] , S)
    df_opc['Bin10'] = ps_to_pl(df['Bin10'] , S)
    df_opc['Bin11'] = ps_to_pl(df['Bin11'] , S)
    df_opc['Bin12'] = ps_to_pl(df['Bin12'] , S)
    df_opc['Bin13'] = ps_to_pl(df['Bin13'] , S)
    df_opc['Bin14'] = ps_to_pl(df['Bin14'] , S)
    df_opc['Bin15'] = ps_to_pl(df['Bin15'] , S)
    df_opc['Bin16'] = ps_to_pl(df['Bin16'] , S)
    df_opc['Bin17'] = ps_to_pl(df['Bin17'] , S)
    df_opc['Bin18'] = ps_to_pl(df['Bin18'] , S)
    df_opc['Bin19'] = ps_to_pl(df['Bin19'] , S)
    df_opc['Bin20'] = ps_to_pl(df['Bin20'] , S)
    df_opc['Bin21'] = ps_to_pl(df['Bin21'] , S)
    df_opc['Bin22'] = ps_to_pl(df['Bin22'] , S)
    df_opc['Bin23'] = ps_to_pl(df['Bin23'] , S)
    
    return df_opc
# --------------------------------------------------------------------------------------------------

def coeficients(grimm,opc):     # Funcion para obtener los coeficientes del cambio de bins opc/grimm, como argumentos lleva los intervalos de memdida de los sensores en una lista
    C = []
    for k in range(len(opc)):
        for i in range(len(grimm)):
            if grimm[i] < opc[k] < grimm[i+1]:
                c = abs(grimm[i+1] - opc[k]) / abs(grimm[i+1] - grimm[i])
                C.append(c)
    return C 
# -------------------------------------------------------------------------

def correct_bins(grimm):   
    
    grim = (0.298,0.352,0.414,0.488,0.576,0.679,0.8,0.943,1.112,1.310,1.545,1.821,2.146,2.53,2.982,3.515,4,144,4.884,5.757,6.787,8,9.43,11.12)
    opc = (0.35,0.46,0.66,1,1.3,1.7,2.3,3,4,5.2,6.5,8,10)

    c = coeficients(grim, opc)
    
    grimm_to_opc = pd.DataFrame()

    bin00 = []
    bin01 = []
    bin02 = []
    bin03 = []
    bin04 = []
    bin05 = []
    bin06 = []
    bin07 = []
    bin08 = []
    bin09 = []
    bin10 = []
    bin11 = []

    for k in range(len(df_opc['Bin00'])):
        bin00.append( c[0]*grimm.iloc[k,:][1] + grimm.iloc[k,:][2] + (1-c[1])*grimm.iloc[k,:][3] )
        bin01.append( c[1]*grimm.iloc[k,:][3] + grimm.iloc[k,:][4] + (1-c[2])*grimm.iloc[k,:][5] )
        bin02.append( c[2]*grimm.iloc[k,:][5] + grimm.iloc[k,:][6] + grimm.iloc[k,:][7] + (1-c[3])*grimm.iloc[k,:][8] )   
        bin03.append( c[3]*grimm.iloc[k,:][8] + (1-c[4])*grimm.iloc[k,:][9] )
        bin04.append( c[4]*grimm.iloc[k,:][9] + grimm.iloc[k,:][10] + (1-c[5])*grimm.iloc[k,:][11] )
        bin05.append( c[5]*grimm.iloc[k,:][11] + grimm.iloc[k,:][12] + (1-c[6])*grimm.iloc[k,:][13] )
        bin06.append( c[6]*grimm.iloc[k,:][13] + grimm.iloc[k,:][14] +(1-c[7])*grimm.iloc[k,:][15] )
        bin07.append( c[7]*grimm.iloc[k,:][15] + (1-c[8])*grimm.iloc[k,:][16] )
        bin08.append( c[8]*grimm.iloc[k,:][16] + grimm.iloc[k,:][17] + (1-c[9])*grimm.iloc[k,:][18])
        bin09.append( c[9]*grimm.iloc[k,:][18] + (1-c[10])*grimm.iloc[k,:][19])
        bin10.append( c[10]*grimm.iloc[k,:][19] + grimm.iloc[k,:][20] )
        bin11.append( grimm.iloc[k,:][21] + (1-c[11])*grimm.iloc[k,:][22] )
    
    grimm_to_opc['date'] = grimm.index
    grimm_to_opc['Bin00_Reference'] = bin00
    grimm_to_opc['Bin01_Reference'] = bin01
    grimm_to_opc['Bin02_Reference'] = bin02
    grimm_to_opc['Bin03_Reference'] = bin03
    grimm_to_opc['Bin04_Reference'] = bin04
    grimm_to_opc['Bin05_Reference'] = bin05
    grimm_to_opc['Bin06_Reference'] = bin06
    grimm_to_opc['Bin07_Reference'] = bin07
    grimm_to_opc['Bin08_Reference'] = bin08
    grimm_to_opc['Bin09_Reference'] = bin09
    grimm_to_opc['Bin10_Reference'] = bin10
    grimm_to_opc['Bin11_Reference'] = bin11
    
    return grimm_to_opc